CERCO DI OTTENERE LA DATA DI NASCITA DEGLI ARTISTI

In [3]:
import pandas as pd
import wikipedia
import re
import concurrent.futures
from tqdm import tqdm

# Imposta la lingua italiana e attiva il rate limiting
wikipedia.set_lang("it")
wikipedia.set_rate_limiting(True)

# Carica il file
file_path = "File Unico Sanremo.xlsx"
df = pd.read_excel(file_path, sheet_name="Artisti Unici con genere")

# Cache per evitare richieste duplicate
dob_cache = {}

# Funzione per determinare la data di nascita
def estrai_data_nascita(artista):
    if artista in dob_cache:
        return dob_cache[artista]

    # Se contiene multipli artisti, salta
    if any(sep in artista.lower() for sep in [",", " e ", " - "]):
        print(f"⚠️ Nomi multipli rilevati, salto: {artista}")
        return "Da verificare"

    try:
        print(f"🟨 Estrazione data di nascita per: {artista}")
        # Timeout gestito internamente da wikipedia API wrapper
        content = wikipedia.page(artista, auto_suggest=False, preload=False).content.lower()
    except (wikipedia.exceptions.DisambiguationError, wikipedia.exceptions.PageError, wikipedia.exceptions.HTTPTimeoutError, Exception) as e:
        print(f"❌ Errore Wikipedia su: {artista} — {e}")
        return "Da verificare"

    # Pattern per date tipo "nato il 3 aprile 1980" o "nata il 25 dicembre 1975"
    match = re.search(r"(nato|nata) il (\d{1,2}) ([a-z]+) (\d{4})", content)
    if match:
        giorno = match.group(2).zfill(2)
        mese = match.group(3)
        anno = match.group(4)
        data = f"{giorno} {mese} {anno}"
        dob_cache[artista] = data
        return data
    else:
        print(f"⚠️ Nessuna data trovata per: {artista}")
        return "Da verificare"

# Funzione per applicare in parallelo con barra di avanzamento
def apply_parallel(df, func, descrizione):
    results = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(func, artista): artista for artista in df["Artista"]}
        for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc=descrizione):
            artista = futures[future]
            try:
                result = future.result()
                results.append(result)
            except Exception as e:
                print(f"❌ Errore durante {descrizione} su: {artista} — {e}")
                results.append("Errore")
    return results

# Applica la funzione
df["Data di nascita"] = apply_parallel(df, estrai_data_nascita, "Estrazione date")

# Salva il nuovo file
output_file = "Artisti_unici_con_data_nascita_timeout.xlsx"
df.to_excel(output_file, index=False)

print(f"\n✅ File salvato con successo: {output_file}")


🟨 Estrazione data di nascita per: Achille Togliani
🟨 Estrazione data di nascita per: Oscar Carboni
🟨 Estrazione data di nascita per: Gino Latilla
🟨 Estrazione data di nascita per: Zucchero
🟨 Estrazione data di nascita per: Will
🟨 Estrazione data di nascita per: Virginio
🟨 Estrazione data di nascita per: Ultimo
🟨 Estrazione data di nascita per: Tricarico
🟨 Estrazione data di nascita per: Tiziano Orecchio
🟨 Estrazione data di nascita per: Stefano Tosi


Estrazione date:   0%|          | 1/1060 [00:00<09:39,  1.83it/s]

❌ Errore Wikipedia su: Tiziano Orecchio — Page id "Tiziano Orecchio" does not match any pages. Try another id!
🟨 Estrazione data di nascita per: Ugo Molinari


Estrazione date:   0%|          | 4/1060 [00:01<03:36,  4.88it/s]

⚠️ Nessuna data trovata per: Achille Togliani
🟨 Estrazione data di nascita per: Gianni Marzocchi
⚠️ Nessuna data trovata per: Virginio
🟨 Estrazione data di nascita per: Stefano Rosso
⚠️ Nessuna data trovata per: Gino Latilla
🟨 Estrazione data di nascita per: Silvano Vittorio
⚠️ Nessuna data trovata per: Tricarico
🟨 Estrazione data di nascita per: Sangiovanni
⚠️ Nessuna data trovata per: Oscar Carboni
🟨 Estrazione data di nascita per: Samuel
❌ Errore Wikipedia su: Stefano Tosi — "Stefano Tosi" may refer to: 
Stefano Tosi
Stefano Tosi
🟨 Estrazione data di nascita per: Roberto Carlos
❌ Errore Wikipedia su: Will — "Will" may refer to: 
Will
Will
Will
Will
Will
Will
Brad Will
Will
WiLL
🟨 Estrazione data di nascita per: Random


...

Estrazione date:  98%|█████████▊| 1043/1060 [01:03<00:00, 108.07it/s]

⚠️ Nessuna data trovata per: Aguaviva
⚠️ Nomi multipli rilevati, salto: Mina - Betty Curtis
⚠️ Nomi multipli rilevati, salto: Wilma De Angelis - Gloria Christian
⚠️ Nomi multipli rilevati, salto: Mina - Nelly Fioramonti
⚠️ Nomi multipli rilevati, salto: Mina - Jenny Luna
⚠️ Nomi multipli rilevati, salto: Aura D'Angelo - Silvia Guidi
⚠️ Nomi multipli rilevati, salto: Nadia Liani - Jolanda Rossin
⚠️ Nomi multipli rilevati, salto: Milva - Miriam Del Mare
⚠️ Nomi multipli rilevati, salto: Tonina Torrielli - Nelly Fioramonti
⚠️ Nomi multipli rilevati, salto: Silvia Guidi - Jenny Luna
⚠️ Nomi multipli rilevati, salto: Wanda Romanelli - Nelly Fioramonti
⚠️ Nomi multipli rilevati, salto: Wilma De Angelis - Tanya
⚠️ Nomi multipli rilevati, salto: Wilma De Angelis - Lucia Altieri
⚠️ Nomi multipli rilevati, salto: Nilla Pizzi e Doppio Quintetto Vocale - Teddy Reno e Quartetto Stars
⚠️ Nomi multipli rilevati, salto: Claudio Villa - Eugenia Foligatti
⚠️ Nomi multipli rilevati, salto: Tonina Torriel

Estrazione date: 100%|██████████| 1060/1060 [01:04<00:00, 16.50it/s] 

⚠️ Nessuna data trovata per: Cugini di Campagna



✅ File salvato con successo: Artisti_unici_con_data_nascita_timeout.xlsx
